In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import voeventdb
import sqlalchemy
import voeventparse as vp

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
from voeventdb.tests.resources import swift_bat_grb_pos_v2_etree
from voeventdb.database.models import Voevent, Base
from voeventdb.database import session_registry as db_session
from voeventdb.tests.config import admin_db_url, testdb_scratch_url, testdb_corpus_url
row = Voevent.from_etree(swift_bat_grb_pos_v2_etree)

In [ ]:
db_url = testdb_corpus_url

In [ ]:
db_session.configure(bind=sqlalchemy.engine.create_engine(testdb_corpus_url))

In [ ]:
s = db_session()
s.query(Voevent).count()

In [ ]:
s.rollback()

In [ ]:
# import voeventdb.tests.fixtures.fake as fake
# from datetime import datetime, timedelta
# packets = fake.heartbeat_packets()
# s.add_all((Voevent.from_etree(p) for p in packets))
# s.commit()
# s.query(Voevent).count()

In [ ]:
s = db_session()
s.query(Voevent).count()